### Scarichiamo le principali librerie che ci serviranno:

In [56]:
%pip install selenium
%pip install webdriver-manager
%pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\burre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\burre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requests in c:\users\burre\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages (2.31.0)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\burre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Importiamo le librerie da usare:

In [57]:
import os
import re
import sys
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import locale
import datetime
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from lxml import etree


### Inizializziamo l'ambiente per lo scraping:

In [58]:
# Imposta in italiano la formattazione della data
locale.setlocale(locale.LC_TIME, 'it_IT.UTF-8')

# Percorso del Desktop dell'utente
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')

# Percorso della cartella Dataset sul Desktop
dataset_folder = os.path.join(desktop_path, 'Dataset')

# Percorso del file NormattivaLinks
normattiva_links_file = os.path.join(os.path.expanduser('~'), 'Desktop', 'NormattivaLinks.txt')

# Controlla se il file esiste, se no lo crea
if not os.path.isfile(normattiva_links_file):
    open(normattiva_links_file, 'w').close()

# Verifica se la cartella "Dataset" esiste
if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)
    print(f"Cartella 'Dataset' creata in {dataset_folder}")
else:
    print(f"Cartella 'Dataset' già presente in {dataset_folder}")

Cartella 'Dataset' già presente in C:\Users\burre\Desktop\Dataset


In [59]:
# Impostazioni del browser Firefox
options = Options()
options.add_argument("--headless")  # Forza la modalità headless (no GUI)
service = Service()

# Configura le preferenze per il download automatico in Firefox
profile = webdriver.FirefoxProfile()  # Crea un profilo per Firefox
profile.set_preference("browser.download.folderList", 2)  # Imposta la cartella personalizzata
profile.set_preference("browser.download.manager.showWhenStarting", False)  # Nascondi la finestra di download
profile.set_preference("browser.download.dir", dataset_folder)  # Cartella di download personalizzata
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/octet-stream")  # Tipi di file per il download automatico

# Aggiungi il profilo a Firefox
options.profile = profile

# Intestazioni per simulare una richiesta da un browser reale (Firefox)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
}

driver = None

def start_or_reset_session():
    global driver
    # Se la sessione è già avviata, prova a chiuderla
    if driver is not None:
        try:
            driver.quit()  # Prova a chiudere la sessione esistente
            print("Sessione esistente chiusa.")
        except WebDriverException:
            print("Nessuna sessione aperta da chiudere.")
    
    # Inizia una nuova sessione
    driver = webdriver.Firefox(service=service, options=options)
    print("Nuova sessione avviata.")

### Funzioni per estrarre le informazioni dato il link di Normattiva

In [60]:
# Namespace per lavorare sul file XML
namespaces = {
    'akn': 'http://docs.oasis-open.org/legaldocml/ns/akn/3.0',
    'eli': 'http://data.europa.eu/eli/ontology#',
    'fo': 'urn:oasis:names:tc:opendocument:xmlns:xsl-fo-compatible:1.0',
    'gu': 'http://www.gazzettaufficiale.it/eli/',
    'html': 'http://www.w3.org/1999/xhtml',
    'na': 'http://www.normattiva.it/eli/',
    'nakn': 'http://normattiva.it/akn/vocabulary',
    'nrdfa': 'http://www.normattiva.it/rdfa/',
    'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
    'rdfa': 'http://www.w3.org/1999/xhtml#'
}

# Funzione per correggere il link se manca il protocollo (http o https)
def correct_link_scheme(url, base_url):
    if not url.startswith("http://") and not url.startswith("https://"):
        url = urljoin(base_url, url)
    return url

# Funzione per estrarre i dettagli dalla parte urn:nir del link
def extract_details_from_urn(href):
    # Estrai l'URN dal link
    urn_start = href.find("urn:nir:")
    if urn_start != -1:
        # Prendi tutto ciò che c'è dopo "urn:nir:"
        urn = href[urn_start:]  # Prendi tutto il resto della stringa a partire da "urn:nir:"
        print(f"URN estratto: {urn}")
    
    parts = urn.split(':')

    if len(parts) >= 5:
        tipo_documento = parts[3].replace('.', ' ').upper()  # Decreto o altro
        data_documento, numero_documento = parts[4].split(';')  # data e numero separati da ;
        date_obj = datetime.datetime.strptime(data_documento, '%Y-%m-%d')
        formatted_date = date_obj.strftime('%d %B %Y')    

        return tipo_documento, formatted_date, numero_documento

    return None, None, None
    
# Funzione per trovare il file più recente nella cartella Dataset
def find_most_recent_file(folder_path):
    files = os.listdir(folder_path)

    if not files:
        return None

    most_recent_file = max(
        (os.path.join(folder_path, f) for f in files),
        key=os.path.getmtime
    )
    return most_recent_file

# Funzione per aspettare il file xml e nel caso returnare il file se trovato
def wait_for_recent_xml(dataset_folder):
    recent_file = None
    dot_count = 0  # Contatore dei puntini
    
    while not recent_file or not recent_file.endswith('.xml'):
        # Stampa la riga con i puntini e aggiorna la stessa riga
        print(f"\rAttesa del completamento del download per il file XML{'.' * dot_count}", end="")
        
        # Incrementa il numero di puntini fino a 3, poi resetta
        dot_count = (dot_count + 1) % 4
        time.sleep(1)
        
        # Verifica il file più recente
        recent_file = find_most_recent_file(dataset_folder)
    
    # Stampa il messaggio finale una volta trovato il file
    print(f"\nFile XML trovato: {recent_file}")
    return recent_file

# Funzione per estrarre le informazioni dall'URL e creare la cartella principale
def extract_info_from_url(href):
    
    tipo_documento, formatted_date, numero_documento = extract_details_from_urn(href)
    
    if tipo_documento and formatted_date and numero_documento:
        # Formatta la cartella principale
        folder_name = f"{tipo_documento} {formatted_date},n. {numero_documento}"

        # Leggi il contenuto attuale del file per controllare duplicati
        with open(normattiva_links_file, 'r') as f:
            existing_links = f.readlines()
        
        # Verifica se il link è già presente
        new_entry = f"{href};{folder_name}\n"
        if new_entry not in existing_links:
            # Aggiungi il link e il nome della cartella al file NormattivaLinks
            with open(normattiva_links_file, 'a') as f:
                f.write(new_entry)
        
        
        main_folder = os.path.join(os.path.expanduser('~'), 'Desktop', 'Dataset', folder_name)
        
        if os.path.exists(main_folder):
            print(f"La cartella {main_folder} esiste già. Interrompo l'operazione.")
            return None, None
        else:
            print(f"Cartella da creare: {folder_name}")
            # Crea la cartella principale per il documento
            os.makedirs(main_folder, exist_ok=True)
            return main_folder,folder_name

    else:
        print("Impossibile estrarre le informazioni dall'URL.")
        return None, None

### Funzioni per salvare il contenuto del link di Normattiva:

In [61]:
def extract_text_from_p(p):
    text = ""

    # Aggiungi il testo iniziale dell'elemento <p> se esiste
    if p.text:
        text += p.text.replace('\n', ' ') + " "
    
    # Itera sugli elementi figli di <p> per gestire <ins> e <ref>
    for element in p.iter():
        if element.tag == '{http://docs.oasis-open.org/legaldocml/ns/akn/3.0}ins':
            text += (element.text or "").replace('\n', ' ') + " "
            if element.tail:
                text += element.tail.replace('\n', ' ') + " "
        elif element.tag == '{http://docs.oasis-open.org/legaldocml/ns/akn/3.0}ref':
            text += (element.text or "").replace('\n', ' ') + " "
            if element.tail:
                text += element.tail.replace('\n', ' ') + " "
    
    if p.tail:
        text += p.tail.replace('\n', ' ') + " "
    
    return text.strip()  # Rimuove eventuali spazi iniziali o finali

# Funzione per analizzare e salvare su file .txt la normativa
def analyze_file(file_path,folder_path):
    
    # Verifica che il file sia accessibile e attendi se necessario
    retries = 3
    while retries > 0:
        if os.access(file_path, os.R_OK):  # Verifica se il file è leggibile
            break
        else:
            print(f"File non accessibile, tentativo in corso. Tentativi rimanenti: {retries}")
            time.sleep(2)  # Attendi un po' prima di riprovare
            retries -= 1
    if retries == 0:
        print("Impossibile accedere al file dopo diversi tentativi.")
        return
    # Carica e analizza il file XML
    try:
        tree = etree.parse(file_path)
    except OSError as e:
        print(f"Errore durante la lettura del file '{file_path}': {e}")
        return
    root = tree.getroot()
    # Trova tutti gli elementi <article> all'interno del file XML
    articles = root.findall('.//akn:article', namespaces)

    # print(f"Numero di articoli trovati: {len(articles)}")

    # Estrai e scrivi il testo di ciascun comma (paragrafo) all'interno degli articoli
    for article_index, article in enumerate(articles, start=1):

        article_eid = article.get('eId')

        # print(f"{article_eid}")

        article_folder = os.path.join(folder_path, article_eid)
        os.makedirs(article_folder, exist_ok=True)

        # Trova tutti gli elementi <paragraph> all'interno dell'articolo
        paragraphs = article.findall('.//akn:paragraph', namespaces)
        paragraph_counter = 1  # Inizializza il contatore dei paragrafi

        for paragraph in paragraphs:
            paragraph_eid = paragraph.get('eId')

            # Controlla se il paragrafo ha un elemento <num>
            num = paragraph.find('.//akn:num', namespaces)
            if num is None:
                continue

            paragraph_text = ""

            # Verifica se è una lista o no
            lists = paragraph.findall('.//akn:list', namespaces)
            
            # Se è una lista, salva il contenuto in maniera diversa
            if lists:
                for lst in lists:
                    list_text = ""
                    intro = lst.find('.//akn:intro', namespaces)
                    if intro is not None:
                        ps = intro.findall('.//akn:p', namespaces)
                        for p in ps:
                            list_text += extract_text_from_p(p)
                    points = lst.findall('.//akn:point', namespaces)
                    for point in points:
                        point_num = point.find('.//akn:num', namespaces)
                        point_content = point.find('.//akn:content', namespaces)
                        if point_num is not None:
                            list_text += (point_num.text or "") + " "
                        if point_content is not None:
                            ps = point_content.findall('.//akn:p', namespaces)
                            for p in ps:
                                list_text += extract_text_from_p(p)
                    paragraph_text += list_text
            else:
                # Trova tutti gli elementi <p> all'interno del <content> del paragrafo
                content = paragraph.find('.//akn:content', namespaces)
                if content is not None:
                    ps = content.findall('.//akn:p', namespaces)
                    for p in ps:
                        paragraph_text += extract_text_from_p(p)

            # Se il paragrafo non è vuoto, scrivi il testo completo nel file
            if paragraph_text.strip():
                file_name = f'comma_{paragraph_counter}.txt'
                with open(os.path.join(article_folder, file_name), 'w', encoding='utf-8') as file:
                    file.write(paragraph_text.strip() + '\n')
                paragraph_counter += 1  # Incrementa il contatore solo se il file è stato creato

        # Trova tutti gli elementi <authorialNote> all'interno dell'articolo
        authorial_notes = article.findall('.//akn:authorialNote', namespaces)
        
        for note_index, note in enumerate(authorial_notes, start=1):
            note_text = ""
            note_ps = note.findall('.//akn:p', namespaces)
            
            for p in note_ps:
                note_text += extract_text_from_p(p)
            
            # Scrivi la nota in un file separato
            note_file_name = f'nota_articolo_{note_index}.txt'
            with open(os.path.join(article_folder, note_file_name), 'w', encoding='utf-8') as note_file:
                note_file.write(note_text.strip() + '\n')

### Funzioni per ricerca e download di riferimenti dentro i vari articoli (ricorsivo)

In [ ]:
# Dizionario globale per salvare href senza "~art" e la relativa profondità
global_links_dict = {}

counter_reference = 1  # Variabile globale inizializzata a 1

# Funzione per salvare il contenuto di un articolo in un file .txt
def save_text_to_file(text, folder, content_reference):
    content_reference = content_reference.replace("/", "-")
    global counter_reference  # Dichiara che si sta usando la variabile globale
    file_path = os.path.join(folder, f"{content_reference}_reference_{counter_reference}.txt")
    
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)
    print(f"Contenuto salvato in: {file_path}")
    counter_reference += 1  # Incrementa il contatore ogni volta che un file viene salvato

# Funzione per eseguire la ricerca ricorsiva dei link all'interno degli articoli
def recursive_find(href, folder, depth, max_depth, content_reference):
    if depth > max_depth:
        return  # Se la profondità supera il massimo, fermati

    # Fai la richiesta HTTP per ottenere l'HTML della pagina
    response = requests.get(href)
    if response.status_code != 200:
        print(f"Errore nel caricamento della pagina: {href}")
        return

    # Usa BeautifulSoup per analizzare l'HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Estrai il div 'bodyTesto' principale
    body_testo_div = soup.find('div', class_="bodyTesto")
    if body_testo_div:
        print(f"{folder}")
        article_name = folder.split("/")[-1]  # Estrai il nome dell'articolo
        print(f"\nEstrazione contenuti per l'articolo: {article_name}")

        # Salva il testo dell'articolo
        save_text_to_file(content_reference + "\n" + body_testo_div.get_text(strip=True), folder, content_reference)
        
        # Trova tutti i link nel bodyTesto che contengono "~art"
        href_links = [{'href': a['href'], 'content': a.get_text(strip=True)} for a in body_testo_div.find_all('a', href=True)] 
        print(f"Trovati {len(href_links)} link nell'articolo {article_name}")

        # Per ogni link, chiamerai ricorsivamente la funzione
        for link in href_links:
            if "~art" not in link['href']:
                # Aggiungi il link al dizionario globale, con profondità calcolata come max_depth - depth
                global_links_dict[link['href']] = max_depth - depth
                print(f"Link non contiene ~art. ---------------------------> {link['href']}")
            else:
                full_link = urljoin(href, link['href'])  # Costruisci il link completo
                recursive_find(full_link, folder, depth + 1, max_depth, link['content'])

# Funzione per trovare i riferimenti all'interno della pagina principale
def find_reference(href, main_folder, max_depth):
    global counter_reference  # Dichiara che si sta usando la variabile globale

    # Carica la pagina principale (href) con Selenium per la navigazione iniziale
    driver.get(href)
    time.sleep(2)  # Attendi il caricamento della pagina

    # Ottieni il contenuto della pagina
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Trova il div con id "albero"
    albero_div = soup.find('div', id="albero")
    if not albero_div:
        print("Div con id 'albero' non trovato.")
        return

    # Trova il primo ul all'interno del div
    ul = albero_div.find('ul')
    if not ul:
        print("Nessun <ul> trovato all'interno del div 'albero'.")
        return

    # Lista per salvare i dati estratti
    lista_articoli = []

    # Trova ogni <li> all'interno dell'<ul> e controlla se contiene un <a class="numero_articolo">
    for li in ul.find_all('li'):
        a_tag = li.find('a', class_="numero_articolo")
        if a_tag and 'onclick' in a_tag.attrs:
            # Estrai il link dall'attributo onclick
            onclick_content = a_tag['onclick']
            match = re.search(r"return showArticle\('([^']+)", onclick_content)
            if match:
                link_articolo = urljoin("https://www.normattiva.it", match.group(1))

                # Format articolo e aggiungi alla lista
                articolo_formattato = a_tag.text.lower().replace(" ", "-")
                lista_articoli.append({'link': link_articolo, 'articolo': articolo_formattato})

    # Per ogni articolo nella lista, usa la finestra secondaria per raccogliere i link e avviare la ricerca
    for articolo in lista_articoli:

        article_folder = os.path.join(main_folder, f"art_{articolo['articolo']}")
        counter_reference = 1  # Resetta il contatore all'inizio di ogni articolo

        # Apre una nuova finestra (popup) per ogni articolo
        driver.execute_script("window.open('');")
        # Ottieni l'handle della finestra appena aperta
        window_handles = driver.window_handles
        driver.switch_to.window(window_handles[-1])  # Passa alla nuova finestra aperta

        # Usa la finestra secondaria per caricare la pagina dell'articolo
        driver.get(articolo['link'])
        time.sleep(2)  # Attendi il caricamento della pagina

        # Ottieni il contenuto dell'articolo
        soup_articolo = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Estrai i link dal bodyTesto dell'articolo e inizia la ricerca ricorsiva
        body_testo_div = soup_articolo.find('div', class_="bodyTesto")
        if body_testo_div:
            print(f"\nInizio ricerca ricorsiva per l'articolo: {articolo['articolo']}")
            # Trova tutti i link dentro il bodyTesto e avvia la funzione ricorsiva per ciascun link
            href_links = [{'href': a['href'], 'content': a.get_text(strip=True)} for a in body_testo_div.find_all('a', href=True)] 
            
            for link in href_links:
                if "~art" not in link['href']:
                    # Aggiungi il link al dizionario globale, con profondità calcolata come max_depth - depth
                    global_links_dict[link['href']] = 0
                    print(f"Link non contiene ~art. ---------------------------> {link['href']}")
                else:
                    full_link = urljoin("https://www.normattiva.it", link['href'])
                    recursive_find(full_link, article_folder, 0, max_depth, link['content'])
        
        # Chiudi la finestra secondaria e torna alla finestra principale
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

# Esegui la funzione di ricerca a partire dall'URL di esempio
# main_folder = os.path.join(dataset_folder, "DECRETO DEL PRESIDENTE DELLA REPUBBLICA 22 settembre 1988,n. 447")
# start_or_reset_session()
# href = "https://www.normattiva.it/atto/caricaDettaglioAtto?atto.dataPubblicazioneGazzetta=1988-10-24&atto.codiceRedazionale=088G0492&atto.articolo.numero=0&atto.articolo.sottoArticolo=1&atto.articolo.sottoArticolo1=0&qId=dc09941c-04e8-4748-aef3-ee1d715c3848&tabID=0.9183034699890468&title=lbl.dettaglioAtto"
# find_reference(href, main_folder, 2)

### Funzioni per scaricare il contenuto del link di Normattiva quando non è presente il file akn

In [ ]:
# Funzione per formattare il numero articolo, rimpiazzando spazi con trattini.
def format_article_number(text):
    return text.lower().replace(" ", "-")

# Funzione per salvare il contenuto di un paragrafo in un file
def save_main_text_to_file(main_text, article_folder):
    file_name = f'main_text.txt'
    with open(os.path.join(article_folder, file_name), 'w', encoding='utf-8') as file:
        file.write(main_text.strip() + '\n')
    print(f"Salvato {file_name} in {article_folder}")

# Funzione per cercare gli articoli, aprire ogni link, estrarre e salvare il contenuto
def download_article_without_akn_file(href, main_folder):
    base_url = "https://www.normattiva.it"
    article_links = {}

    # Carica la pagina principale e attende che si carichi completamente
    print(f"Caricamento della pagina principale: {href}")
    driver.get(href)
    time.sleep(1)  # Attendi il caricamento della pagina

    # Analizza la pagina con BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    albero_div = soup.find("div", id="albero")
    if not albero_div:
        print("Errore: div con id 'albero' non trovato.")
        return article_links

    # Trova e filtra gli <li> che contengono un <a> con classe 'numero_articolo'
    list_items = albero_div.find_all("li")
    filtered_items = [
        li for li in list_items 
        if li.find("a", class_="numero_articolo") and 
        not any(x in li.find("a", class_="numero_articolo").text for x in ["orig.", "agg."])
    ]
    
    print(f"Numero di elementi trovati con 'numero_articolo': {len(filtered_items)}")

    # Ottieni l'handle della finestra principale
    main_window = driver.current_window_handle

    for i, li in enumerate(filtered_items):
        article_element = li.find("a", class_="numero_articolo")

        if not article_element or 'onclick' not in article_element.attrs:
            print(f"Elemento {i} saltato: manca un link valido o l'attributo 'onclick'.")
            continue

        # Estrai il testo del numero articolo e formattalo
        article_number_text = article_element.text.strip()
        formatted_article_number = format_article_number(article_number_text)

        # Estrai il parametro dell'URL dall'attributo onclick
        onclick_text = article_element['onclick']
        match = re.search(r"showArticle\('([^']*)'", onclick_text)
        if not match:
            print(f"Elemento {i}: URL non trovato nell'onclick.")
            continue
        
        relative_url = match.group(1)
        article_url = base_url + relative_url
        print(f"Elemento {i} - URL dell'articolo: {article_url}")

        # Chiudi la scheda secondaria se già esistente
        if len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[1])
            driver.close()

        # Apri una nuova scheda e passa a essa
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])  # Passa alla scheda secondaria
        driver.get(article_url)

        # Attendi che il div bodyTesto sia presente
        try:
            body_text_div = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CLASS_NAME, "bodyTesto"))
            )
            article_soup = BeautifulSoup(driver.page_source, 'html.parser')
            body_text_div = article_soup.find("div", class_="bodyTesto")

            article_folder = os.path.join(main_folder, f"art_{formatted_article_number}")
            os.makedirs(article_folder, exist_ok=True)

            # Estrai tutto il testo dal bodyTesto
            full_text = body_text_div.get_text(separator=" ").strip()

            save_main_text_to_file(full_text, article_folder)

        except TimeoutException:
            print(f"Elemento {i}: div con classe 'bodyTesto' non trovato dopo attesa.")
            driver.close()
            driver.switch_to.window(main_window)
            continue

        # Chiudi la scheda secondaria e torna alla finestra principale
        driver.close()
        driver.switch_to.window(main_window)

    print("Download e salvataggio completati per tutti gli articoli.")
    driver.close()


### Funzioni per effettuare il download del contenuto del link:

In [66]:
# Funzione per ottenere il file XML e scaricare il contenuto
def download_from_link(href,main_folder,timeout=20, poll_interval=1):
    try:
        driver.get(href)
        start_time = time.time()
        download_link = None
        
        # Ciclo di polling per cercare l'elemento senza bloccare il codice
        while (time.time() - start_time) < timeout:
            download_buttons = driver.find_elements(By.XPATH, "//a[contains(@href, '/do/atto/caricaAKN?')]")
            if download_buttons:
                download_link = download_buttons[0]
                break
            # Attesa breve prima di riprovare
            time.sleep(poll_interval)
        
        if download_link:
            # Se il pulsante è stato trovato, procedi con il download
            file_url = download_link.get_attribute('href')
            print(f"Link per il download trovato: {file_url}")
            download_link.click()
            
            recent_file = wait_for_recent_xml(dataset_folder)
            if recent_file:
                analyze_file(recent_file, main_folder)
                # Elimina il file XML scaricato dopo averlo elaborato
                os.remove(recent_file)
                print(f"File XML eliminato: {recent_file}")
                
        else:
            # Se il pulsante non è stato trovato entro il tempo limite, esegui la funzione alternativa
            print("Non c'è il pulsante per il download. Provo con un altro metodo.")
            download_article_without_akn_file(href, main_folder)
            
        
    finally: 
        print("Download completato")

# Funzione per scaricare il contenuto di una pagina di Normattiva durante una ricerca ricorsiva
def download_normattiva_content(href, max_depth):
    # Estrai l'URN dal link
    main_folder,folder_name = extract_info_from_url(href)
    try:
        if main_folder and folder_name is not None:
            download_from_link(href,main_folder)
            find_reference(href, main_folder, max_depth)
        else:
            print("Impossibile estrarre le informazioni dall'URL.")
    finally: 
        print("Download primario completato")

### Cerchiamo la norma e la scarichiamo:

In [68]:
MAX_DEPTH = 2

start_or_reset_session()

search_query = input("Inserisci il nome della normativa da cercare: ")
query = search_query + " normattiva"
url = f"https://www.google.com/search?q={query.replace(' ', '+')}"

# Invia la richiesta HTTP alla pagina di ricerca di Google
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Trova tutti i risultati di ricerca
links = soup.find_all('a', href=True)

# Base URL
base_url = "https://www.google.com"

# Cerca il primo link con "uri-res/N2Ls?"
for link in links:
    href = link['href']
    
    if "uri-res/N2Ls?" in href:
        href = urljoin(base_url, href)
        print(f"Link trovato: {href}")
        download_normattiva_content(href, MAX_DEPTH)
        while global_links_dict:
            link, depth = global_links_dict.popitem()
            print(f"Processando link senza ~art: {link} con profondità {depth}")
            download_normattiva_content(link, MAX_DEPTH-depth)
        break
    else:
        print("Nessun link di download trovato.")

driver.quit()

Sessione esistente chiusa.
Nuova sessione avviata.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Nessun link di download trovato.
Link trovato: https://www.normattiva.it/uri-res/N2Ls?urn:nir:stato:decreto.legislativo:2024-07-30;116
URN estratto: urn:nir:stato:decreto.legislativo:2024-07-30;116
Cartella da creare: DECRETO LEGISLATIVO 30 luglio 2024,n. 116
Link per il download trovato: https://www.normattiva.it/do/atto/caricaAKN?dataGU=20240813&codiceRedaz=24G00129&dataVigenza=20241115
Attesa del completamento del 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\burre\\Desktop\\Dataset\\DECRETO LEGISLATIVO 30 luglio 2024,n. 116\\art_1\\art. 8 del D.L.C.P.S. n. 1577/1947_reference_711.txt'